In [1]:
import sys
from pathlib import Path

import numpy as np
import tensorflow as tf

SCRIPT_DIR = Path.cwd()
sys.path.append(str(SCRIPT_DIR.parent))

from data_utils.data_iterator import DataIterator

[[0 0]
 [1 1]
 [2 2]
 [3 3]]
(0, array([[1, 1],
       [0, 0],
       [2, 2]]), array([['b'],
       ['a'],
       ['c']], dtype='<U1'))
(1, array([[3, 3]]), array([['d']], dtype='<U1'))
(0, array([[2, 2],
       [0, 0],
       [3, 3]]), array([['c'],
       ['a'],
       ['d']], dtype='<U1'))
(1, array([[1, 1]]), array([['b']], dtype='<U1'))


## Load the processed data

In [2]:
training_data = np.load(open("training_data/mushrooms_training_data.npy", 'rb'))
training_labels = np.load(open("training_data/mushrooms_training_labels.npy", 'rb'))
validation_data = np.load(open("training_data/mushrooms_validation_data.npy", 'rb'))
validation_labels = np.load(open("training_data/mushrooms_validation_labels.npy", 'rb'))
test_data = np.load(open("training_data/mushrooms_test_data.npy", 'rb'))
test_labels = np.load(open("training_data/mushrooms_test_labels.npy", 'rb'))

In [3]:
training_data.shape

(6500, 117)

In [4]:
training_labels.shape

(6500, 1)

## Model hyperparameters and parameters

In [5]:
# Hyperparameters
epochs = 10
learning_rate = 0.001
batch_size = 512

In [6]:
# Parameters
hidden_1_units = 50
hidden_1_activation = tf.nn.relu
hidden_2_units = 20
hidden_2_activation = tf.nn.relu
hidden_3_units = 5
hidden_3_activation = tf.nn.relu

## Create model
### Placeholders

In [7]:
x = tf.placeholder(dtype=tf.float32, shape=[None, training_data.shape[1]], name="inputs")
y = tf.placeholder(dtype=tf.float32, shape=[None, 1], name="labels")

### Create hidden layers and output layer

In [8]:
fully_connected_1 = tf.layers.dense(x, units=hidden_1_units, activation=hidden_1_activation, name="hidden_1")
fully_connected_2 = tf.layers.dense(fully_connected_1, units=hidden_2_units, activation=hidden_2_activation, name="hidden_2")
fully_connected_3 = tf.layers.dense(fully_connected_2, units=hidden_3_units, activation=hidden_3_activation, name="hidden_3")
output_layer = tf.layers.dense(fully_connected_3, units=1, activation=tf.nn.sigmoid, name="output")

### Cost and Optimizer

In [9]:
init = tf.global_variables_initializer()

with tf.variable_scope("Cost_and_Optimizer"):
    loss_op = tf.nn.sigmoid_cross_entropy_with_logits(labels=y, logits=output_layer, name="loss_op")
    cost = tf.reduce_sum(loss_op, name="cost")
    optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
    train = optimizer.minimize(cost)

### Create lists of batchs

In [10]:
training_iterator = DataIterator(training_data, training_labels, batch_size=batch_size)

## Model training

In [11]:
tf.summary.scalar("Cost", cost)
tf.summary.histogram("fully_connected_1", fully_connected_1)
tf.summary.histogram("output_layer", output_layer)

with tf.variable_scope('hidden_1', reuse=True):
    weights = tf.get_variable('kernel')
    
tf.summary.histogram("weights_fully_1", weights)

with tf.Session() as sess:
    sess.run(init)
    merged_summary = tf.summary.merge_all()
    
    # Handle old tensorboard file with same hyperparameters
    tensorboard_job_name = "{}-{}-{}-lr_{}-e_{}-b_{}".format(hidden_1_units, hidden_2_units, hidden_3_units,
                                                             learning_rate, epochs, batch_size)
    tensorboard_log_dir = Path(Path.cwd(), "tensoboard_logs", tensorboard_job_name)
    writer = tf.summary.FileWriter("./tensoboard_logs/{}".format(tensorboard_job_name))
    if len(list(tensorboard_log_dir.iterdir())) > 0:
        for file in list(tensorboard_log_dir.iterdir()):
            file.unlink()
    
    writer.add_graph(tf.get_default_graph())
    
    training_steps = 0
    for e in range(epochs):
        
        for i, training_data_batch, training_labels_batch in training_iterator:
            
            if i%5 == 0:
                
                output, batch_cost = sess.run([output_layer, cost], feed_dict={x: training_data_batch,
                                                                               y: training_labels_batch})
                
                predictions = output > 0.5
                accuracy = np.sum(training_labels_batch == predictions) / training_labels_batch.shape[0]
                
                validation_output, validation_cost = sess.run([output_layer, cost], feed_dict={x: validation_data,
                                                                                               y: validation_labels})
                validation_predictions = validation_output > 0.5
                validation_accuracy = np.sum(validation_labels == validation_predictions) / validation_labels.shape[0]
                
                print("Epoch N°: {} - Batch N°: {}\n"
                      "Training cost = {} - Training accuracy = {}%\n"
                      "Validation cost = {} - Validation accuracy = {}%".format(e, i, batch_cost, accuracy * 100,
                                                                               validation_cost, validation_accuracy * 100))
            
            s = sess.run(merged_summary, feed_dict={x: training_data_batch, y: training_labels_batch})
            writer.add_summary(s, training_steps)
            training_steps += 1
            
            sess.run(train, feed_dict={x: training_data_batch, y: training_labels_batch})

Epoch N°: 0 - Batch N°: 0
Training cost = 365.53662109375 - Training accuracy = 42.578125%
Validation cost = 578.06396484375 - Validation accuracy = 40.02463054187192%
Epoch N°: 0 - Batch N°: 5
Training cost = 366.5616455078125 - Training accuracy = 50.9765625%
Validation cost = 568.5856323242188 - Validation accuracy = 47.53694581280788%
Epoch N°: 0 - Batch N°: 10
Training cost = 344.6744384765625 - Training accuracy = 50.0%
Validation cost = 539.0370483398438 - Validation accuracy = 47.53694581280788%
Epoch N°: 1 - Batch N°: 0
Training cost = 333.656005859375 - Training accuracy = 49.8046875%
Validation cost = 523.320068359375 - Validation accuracy = 47.53694581280788%
Epoch N°: 1 - Batch N°: 5
Training cost = 313.81927490234375 - Training accuracy = 46.2890625%
Validation cost = 502.629150390625 - Validation accuracy = 47.53694581280788%
Epoch N°: 1 - Batch N°: 10
Training cost = 309.36956787109375 - Training accuracy = 67.3828125%
Validation cost = 491.5146484375 - Validation accur

## Tensorboard

To use tensorboard open a console, go to the project folder (`"mushrooms"`) type:

```bash
tensorboard --logdir tensoboard_logs\
```

### Tensoboard results

![output_layer_histograms](output_layer_histograms.png)
![graph](graph.png)